### **1. Data Loading and Preprocessing**

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
import folium
from typing import List, Tuple, Dict, Optional
import random
import time
import matplotlib.pyplot as plt

# Load data
parking_data = pd.read_csv('Parking_Lot_Dataset.csv')

### **2. ParkingOptimizer Class**
1.**__init__**:Initialize the parking optimizer

2.**haversine_distance**:Calculate the distance between two points

3.**_filter_by_distance**:Filter parking lots within distance

4.**calculate_fitness**:Calculate fitness score for a parking lot

5.**initialize_particles**:Initialize PSO particles

6.**optimize**:Run PSO optimization to find best parking lot

7.**linear_baseline**:Select the best parking lot based on a linear combination of weighted factors

8.**visualize_results**:Create a map visualization of parking lots and recommendations




In [6]:
class ParkingOptimizer:
    #Initialize the parking optimizer
    def __init__(self, parking_data: pd.DataFrame, destination: Tuple[float, float],
                 weights: Dict[str, float] = None):

        # Store initial parameters
        self.parking_data = parking_data
        self.destination = destination
        self.weights = weights or {'distance': 0.5, 'cost': 0.3, 'accessibility': 0.2}

        # Calculate distances of parking lots
        self.parking_data['distance'] = self.parking_data.apply(
            lambda row: self.haversine_distance(
                row['lat'], row['lng'],
                self.destination[0], self.destination[1]),
            axis=1
        )
        # PSO algorithm parameters
        self.num_particles = 20
        self.max_iterations = 50
        self.c1 = 2.0
        self.c2 = 2.0
        self.w = 0.7

    def haversine_distance(self, lat1: float, lon1: float, lat2: float, lon2: float) -> float:
        """ Calculate the distance between two points on Earth using Haversine formula. """
        R = 6371000  # Earth's radius in meters

        # Convert decimal degrees to radians
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

        # Haversine formula components
        dlat = lat2 - lat1
        dlon = lon2 - lon1

        # Calculate distance
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * atan2(sqrt(a), sqrt(1-a))
        distance = R * c

        return distance

    def _filter_by_distance(self, radius: float) -> pd.DataFrame:
        """Filter parking lots within specified radius of destination."""
        return self.parking_data[self.parking_data['distance'] <= radius].copy()

    def calculate_fitness(self, parking_lot: pd.Series) -> float:
        """Calculate fitness score for a parking lot."""
        # Get maximum values for normalization
        max_distance = self.filtered_data['distance'].max()
        max_cost = self.filtered_data['rate_hourly'].max()

        # Calculate distance score
        distance_score = (max_distance - parking_lot['distance']) / max_distance if max_distance > 0 else 1

        # Calculate cost score
        cost_score = (max_cost - parking_lot['rate_hourly']) / max_cost if max_cost > 0 else 1

        # Calculate accessibility score
        accessibility_score = random.uniform(0.7, 1.0)

        # Calculate weighted sum of all factors
        fitness = (
            self.weights['distance'] * distance_score +
            self.weights['cost'] * cost_score +
            self.weights['accessibility'] * accessibility_score
        )
        return fitness

    def initialize_particles(self) -> Tuple[List[int], List[float], List[float]]:
        """Initialize PSO particles."""
        # Adjust number of particles
        self.num_particles = min(self.num_particles, len(self.filtered_data))

        # Initialize random positions, velocities, and fitness values
        positions = random.sample(range(len(self.filtered_data)), self.num_particles)
        velocities = [random.uniform(-1, 1) for _ in range(self.num_particles)]
        particle_best_fitness = [self.calculate_fitness(self.filtered_data.iloc[pos])
                                 for pos in positions]
        return positions, particle_best_fitness, velocities

    def optimize(self) -> Optional[Tuple[pd.Series, int, List[float]]]:
        """Run PSO optimization to find best parking lot."""
        # Try 500m radius first
        self.filtered_data = self._filter_by_distance(500)
        if len(self.filtered_data) == 0:
            print("No parking lots are available within an 500-meter radius. Expanding the search radius to 750 meters...")
            # Try 1.5×500=750m radius
            self.filtered_data = self._filter_by_distance(750)
            if len(self.filtered_data) == 0:
                print("No parking lots are available within a 750-meter radius.")
                return None
            search_radius = 750
        else:
            search_radius = 500

        # Initialize PSO
        positions, particle_best_fitness, velocities = self.initialize_particles()
        particle_best_positions = positions.copy()

        # Initialize global best
        global_best_idx = np.argmax(particle_best_fitness)
        global_best_position = positions[global_best_idx]
        global_best_fitness = particle_best_fitness[global_best_idx]

        # For convergence tracking
        convergence_history = [global_best_fitness]

        # Main optimization loop
        for _ in range(self.max_iterations):
            for i in range(self.num_particles):
                # Update velocity
                r1, r2 = random.random(), random.random()
                cognitive = self.c1 * r1 * (particle_best_positions[i] - positions[i])
                social = self.c2 * r2 * (global_best_position - positions[i])
                velocities[i] = self.w * velocities[i] + cognitive + social

                # Update position
                new_position = int(positions[i] + velocities[i]) % len(self.filtered_data)
                positions[i] = new_position

                # Calculate fitness
                fitness = self.calculate_fitness(self.filtered_data.iloc[positions[i]])

                # Update particle best
                if fitness > particle_best_fitness[i]:
                    particle_best_fitness[i] = fitness
                    particle_best_positions[i] = positions[i]

                    # Update global best
                    if fitness > global_best_fitness:
                        global_best_fitness = fitness
                        global_best_position = positions[i]

            # Record global best fitness
            convergence_history.append(global_best_fitness)

        return self.filtered_data.iloc[global_best_position], search_radius, convergence_history

    def linear_baseline(self) -> Optional[pd.Series]:
        """Select the best parking lot based on a linear combination of weighted factors."""
        # Filter parking lots within 500m
        self.filtered_data = self._filter_by_distance(500)
        if len(self.filtered_data) == 0:
            print("No parking lots are available within an 500-meter radius. Expanding the search radius to 750 meters...")
            # Expand to 750m if no parking lots are found
            self.filtered_data = self._filter_by_distance(750)
            if len(self.filtered_data) == 0:
                print("No parking lots are available within a 750-meter radius.")
                return None

        # Calculate fitness scores
        self.filtered_data['fitness'] = self.filtered_data.apply(
            lambda row: self.calculate_fitness(row),
            axis=1
        )

        # Select the parking lot with the highest fitness score
        best_parking_lot = self.filtered_data.loc[self.filtered_data['fitness'].idxmax()]

        return best_parking_lot

    # Map Visualization
    def visualize_results(self, best_parking_lot_pso: Optional[pd.Series],
                          search_radius: int,
                          baseline_parking_lot: Optional[pd.Series] = None) -> folium.Map:
        """Create a map visualization of parking lots and recommendations."""
        # Create map centered on destination
        m = folium.Map(location=[self.destination[0], self.destination[1]],
                       zoom_start=15)

        # Add destination marker
        folium.Marker(
            [self.destination[0], self.destination[1]],
            popup='Destination',
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(m)

        # Add circle showing search radius
        folium.Circle(
            radius=search_radius,
            location=[self.destination[0], self.destination[1]],
            color='red',
            fill=True,
            fillOpacity=0.1
        ).add_to(m)

        # Add markers for parking lots within search radius
        for _, lot in self.filtered_data.iterrows():
            if best_parking_lot_pso is not None and lot['id'] == best_parking_lot_pso['id']:
                continue
            if baseline_parking_lot is not None and lot['id'] == baseline_parking_lot['id']:
                continue

            popup_text = f"""
                ID: {lot['id']}<br>
                Rate: ${lot['rate_hourly']}/hr<br>
                Distance: {lot['distance']:.0f}m
            """
            folium.Marker(
                [lot['lat'], lot['lng']],
                popup=popup_text,
                icon=folium.Icon(color='blue')  # Candidate parking lots
            ).add_to(m)

        # Add marker for baseline parking lot
        if baseline_parking_lot is not None:
            popup_text = f"""
                Baseline Recommendation<br>
                ID: {baseline_parking_lot['id']}<br>
                Rate: ${baseline_parking_lot['rate_hourly']}/hr<br>
                Distance: {baseline_parking_lot['distance']:.0f}m
            """
            folium.Marker(
                [baseline_parking_lot['lat'], baseline_parking_lot['lng']],
                popup=popup_text,
                icon=folium.Icon(color='orange', icon='star')
            ).add_to(m)

        # Add marker for PSO best parking lot
        if best_parking_lot_pso is not None:
            popup_text = f"""
                PSO Recommendation<br>
                ID: {best_parking_lot_pso['id']}<br>
                Rate: ${best_parking_lot_pso['rate_hourly']}/hr<br>
                Distance: {best_parking_lot_pso['distance']:.0f}m
            """
            folium.Marker(
                [best_parking_lot_pso['lat'], best_parking_lot_pso['lng']],
                popup=popup_text,
                icon=folium.Icon(color='green', icon='ok-sign')
            ).add_to(m)

        return m

### **Simulation of Efficiency**

In [7]:
# Define the geographical boundaries of downtown Toronto
# Coordinates:Latitude: 43.6400 to 43.6600 Longitude: -79.3950 to -79.3700

min_lat, max_lat = 43.6400, 43.6600
min_lng, max_lng = -79.3950, -79.3700

# Set the scoring weights
weights = {
    'distance': 0.5,     # 50% weight for distance
    'cost': 0.3,         # 30% weight for cost
    'accessibility': 0.2 # 20% weight for accessibility
}

# List to store efficiency results
efficiency_results = []

# Loop over different seed values to get different destinations
for seed_value in [42, 123, 456]:
    print(f"\n---\nUsing seed value: {seed_value}")

    # Set random seeds
    random.seed(seed_value)
    np.random.seed(seed_value)

    # Generate a random destination within downtown Toronto
    destination = (
        random.uniform(min_lat, max_lat),
        random.uniform(min_lng, max_lng)
    )

    print(f"Randomly selected destination: ({destination[0]:.6f}, {destination[1]:.6f})")

    # Create an instance of the optimizer
    optimizer = ParkingOptimizer(parking_data, destination, weights)

    # Measure execution time for Linear Baseline
    start_time_linear = time.time()
    baseline_parking_lot = optimizer.linear_baseline()
    end_time_linear = time.time()
    linear_time = end_time_linear - start_time_linear

    # Measure execution time for PSO
    start_time_pso = time.time()
    pso_result = optimizer.optimize()
    end_time_pso = time.time()
    pso_time = end_time_pso - start_time_pso

    if pso_result is not None and baseline_parking_lot is not None:
        best_parking_lot_pso, search_radius, convergence_history = pso_result

        # Calculate fitness score for PSO result
        best_parking_lot_pso['fitness'] = optimizer.calculate_fitness(best_parking_lot_pso)

        # Print out details of the baseline and PSO results
        print("\nLinear Baseline Parking Lot:")
        print(f"ID: {baseline_parking_lot['id']}")
        print(f"Location: ({baseline_parking_lot['lat']}, {baseline_parking_lot['lng']})")
        print(f"Hourly Rate: ${baseline_parking_lot['rate_hourly']}")
        print(f"Distance to destination: {baseline_parking_lot['distance']:.0f}m")
        print(f"Fitness Score: {baseline_parking_lot['fitness']:.4f}")
        print(f"Execution Time: {linear_time:.6f} seconds")

        print("\nPSO Optimized Parking Lot:")
        print(f"ID: {best_parking_lot_pso['id']}")
        print(f"Location: ({best_parking_lot_pso['lat']}, {best_parking_lot_pso['lng']})")
        print(f"Hourly Rate: ${best_parking_lot_pso['rate_hourly']}")
        print(f"Distance to destination: {best_parking_lot_pso['distance']:.0f}m")
        print(f"Fitness Score: {best_parking_lot_pso['fitness']:.4f}")
        print(f"Execution Time: {pso_time:.6f} seconds")

        # Visualize results with both PSO and baseline
        map_viz = optimizer.visualize_results(best_parking_lot_pso, search_radius, baseline_parking_lot)
        map_viz.save(f'parking_recommendation_seed_{seed_value}.html')

        # Plot convergence history
        plt.figure()
        plt.plot(convergence_history)
        plt.title(f'PSO Convergence History (Seed {seed_value})')
        plt.xlabel('Iteration')
        plt.ylabel('Global Best Fitness')
        plt.grid(True)
        plt.savefig(f'pso_convergence_seed_{seed_value}.png')
        plt.close()

        # Store efficiency results
        efficiency_results.append({
            'seed': seed_value,
            'linear_fitness': baseline_parking_lot['fitness'],
            'pso_fitness': best_parking_lot_pso['fitness'],
            'linear_time': linear_time,
            'pso_time': pso_time
        })

    else:
        print("No parking lots were found by one or both methods.")
        # If no parking lot is found, create a map showing the search area
        map_viz = optimizer.visualize_results(None, 750)
        map_viz.save(f'parking_recommendation_seed_{seed_value}.html')


---
Using seed value: 42
Randomly selected destination: (43.652789, -79.394375)

Linear Baseline Parking Lot:
ID: 433.0
Location: (43.6512722411769, -79.3909174086614)
Hourly Rate: $0.0
Distance to destination: 325m
Fitness Score: 0.6515
Execution Time: 0.008243 seconds

PSO Optimized Parking Lot:
ID: 433.0
Location: (43.6512722411769, -79.3909174086614)
Hourly Rate: $0.0
Distance to destination: 325m
Fitness Score: 0.6121
Execution Time: 0.017079 seconds


<ipython-input-7-347361031504>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_parking_lot_pso['fitness'] = optimizer.calculate_fitness(best_parking_lot_pso)
<ipython-input-7-347361031504>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_parking_lot_pso['fitness'] = optimizer.calculate_fitness(best_parking_lot_pso)



---
Using seed value: 123
Randomly selected destination: (43.641047, -79.392820)
No parking lots are available within an 500-meter radius. Expanding the search radius to 750 meters...
No parking lots are available within an 500-meter radius. Expanding the search radius to 750 meters...

Linear Baseline Parking Lot:
ID: 47.0
Location: (43.641635, -79.399159)
Hourly Rate: $4.0
Distance to destination: 514m
Fitness Score: 0.4234
Execution Time: 0.003394 seconds

PSO Optimized Parking Lot:
ID: 47.0
Location: (43.641635, -79.399159)
Hourly Rate: $4.0
Distance to destination: 514m
Fitness Score: 0.4181
Execution Time: 0.039442 seconds


<ipython-input-7-347361031504>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_parking_lot_pso['fitness'] = optimizer.calculate_fitness(best_parking_lot_pso)
<ipython-input-7-347361031504>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_parking_lot_pso['fitness'] = optimizer.calculate_fitness(best_parking_lot_pso)



---
Using seed value: 456
Randomly selected destination: (43.654964, -79.370835)

Linear Baseline Parking Lot:
ID: 529.0
Location: (43.6547497448117, -79.3700940045127)
Hourly Rate: $0.0
Distance to destination: 64m
Fitness Score: 0.8976
Execution Time: 0.002072 seconds

PSO Optimized Parking Lot:
ID: 529.0
Location: (43.6547497448117, -79.3700940045127)
Hourly Rate: $0.0
Distance to destination: 64m
Fitness Score: 0.8768
Execution Time: 0.018925 seconds


<ipython-input-7-347361031504>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_parking_lot_pso['fitness'] = optimizer.calculate_fitness(best_parking_lot_pso)
<ipython-input-7-347361031504>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_parking_lot_pso['fitness'] = optimizer.calculate_fitness(best_parking_lot_pso)


### **Efficiency comparison**

In [8]:
# Display the efficiency comparison
print("\nEfficiency Comparison:")
df_efficiency = pd.DataFrame(efficiency_results)
print(df_efficiency)


Efficiency Comparison:
   seed  linear_fitness  pso_fitness  linear_time  pso_time
0    42        0.651498     0.612145     0.008243  0.017079
1   123        0.423358     0.418143     0.003394  0.039442
2   456        0.897625     0.876765     0.002072  0.018925


### **Execution times**

In [12]:
# Plot execution times
plt.figure()
plt.bar(df_efficiency['seed'] - 0.2, df_efficiency['linear_time'], width=0.4, label='Linear Baseline')
plt.bar(df_efficiency['seed'] + 0.2, df_efficiency['pso_time'], width=0.4, label='PSO')
plt.xticks(df_efficiency['seed'])
plt.xlabel('Seed Value')
plt.ylabel('Execution Time (seconds)')
plt.title('Execution Time Comparison')
plt.legend()
plt.grid(True)
plt.savefig('execution_time_comparison.png')
plt.close()

### **Fitness scores**

In [11]:
# Plot fitness scores
plt.figure()
plt.bar(df_efficiency['seed'] - 0.2, df_efficiency['linear_fitness'], width=0.4, label='Linear Baseline')
plt.bar(df_efficiency['seed'] + 0.2, df_efficiency['pso_fitness'], width=0.4, label='PSO')
plt.xticks(df_efficiency['seed'])
plt.xlabel('Seed Value')
plt.ylabel('Fitness Score')
plt.title('Fitness Score Comparison')
plt.legend()
plt.grid(True)
plt.savefig('fitness_score_comparison.png')
plt.close()